In [3]:
%pip install accelerate
%pip install -i https://pypi.org/simple/ bitsandbytes
%pip install transformers[torch] -U

%pip install datasets
%pip install langchain
%pip install langchain_community
%pip install PyMuPDF
%pip install sentence-transformers
%pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple/
Note: you may need to restart the kernel to use updated packages.
  Using cached tokenizers-0.19.1-cp311-none-win_amd64.whl.metadata (6.9 kB)
Using cached tokenizers-0.19.1-cp311-none-win_amd64.whl (2.2 MB)
   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
   ---------------------------------------- 9.4/9.4 MB 65.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
  Using cached datasets-2.20.0-py3-none-any.whl.metadata (19 kB)
  Using cached pyarrow-17.0.0-cp311-cp311-win_amd64.whl.metadata (3.4 kB)
  Using cached pyarrow_hotfix-0.6-py3-none-any.whl.metadata (3.6 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached pandas-2.2.2-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Using cached xxhash-3.4.1-cp311-cp311-win_amd64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py311-none-any.whl.metad

In [1]:
import os
import unicodedata
import torch
import pandas as pd
from tqdm import tqdm
import fitz  # PyMuPDF
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    BitsAndBytesConfig
)
from accelerate import Accelerator
from peft import LoraConfig, PeftModel
# Langchain 관련
from langchain_community.document_loaders import PDFMinerLoader, PyMuPDFLoader, PDFPlumberLoader
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter, MarkdownHeaderTextSplitter
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain.retrievers import ContextualCompressionRetriever


d:\ai_project\chanyoung\dacon\dacon\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pymupdf4llm

def process_langchain_pdf(file_path):
    md_text = pymupdf4llm.to_markdown(file_path)
    
    headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
    ]

    md_header_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on, strip_headers=False)
    md_chunks = md_header_splitter.split_text(md_text)

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=512, chunk_overlap=128
    )

    splits = text_splitter.split_documents(md_chunks)
    return splits

In [3]:
"""def process_langchain_pdf(file_path):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 512,
        chunk_overlap = 32
    )
    docs = []
    
    loader = PDFPlumberLoader(file_path)
    documents = loader.load_and_split(text_splitter=text_splitter)
    docs.extend(documents)
    return docs

def process_pdf(file_path, chunk_size=324, chunk_overlap=126):
   
    # PDF 파일 열기
    doc = fitz.open(file_path)
    text = ''
    # 모든 페이지의 텍스트 추출
    for page in doc:
        text += page.get_text()
    # 텍스트를 chunk로 분할
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,

    )
    chunk_temp = splitter.split_text(text)
    # Document 객체 리스트 생성
    chunks = [Document(page_content=t) for t in chunk_temp]
    return chunks"""

# upskyy/e5-base-korean 5gb BAAI/bge-multilingual-gemma2 : 23GB  intfloat/multilingual-e5-base jhgan/ko-sroberta-multitask BAAI/bge-m3 :16gb
def create_vector_db(chunks, model_path="upskyy/e5-base-korean"):
    """FAISS DB 생성"""
    # 임베딩 모델 설정
    model_kwargs = {'device': 'cuda'}
    encode_kwargs = {'normalize_embeddings': True}
    embeddings = HuggingFaceEmbeddings(
        model_name=model_path,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )
    # FAISS DB 생성 및 반환
    db = FAISS.from_documents(chunks, embedding=embeddings)
    return db

def create_bm25(chunks):
    """bm25_retriever 생성"""
    bm25_retriever = BM25Retriever.from_documents(chunks)
    bm25_retriever.k = 4
    return bm25_retriever

def normalize_path(path):
    """경로 유니코드 정규화"""
    return unicodedata.normalize('NFC', path)


def process_pdfs_from_dataframe(df, base_directory):
    """딕셔너리에 pdf명을 키로해서 DB, retriever 저장"""
    pdf_databases = {}
    unique_paths = df['Source_path'].unique()
    
    for path in tqdm(unique_paths, desc="Processing PDFs"):
        # 경로 정규화 및 절대 경로 생성
        normalized_path = normalize_path(path)
        full_path = os.path.normpath(os.path.join(base_directory, normalized_path.lstrip('./'))) if not os.path.isabs(normalized_path) else normalized_path
        
        pdf_title = os.path.splitext(os.path.basename(full_path))[0]
        print(f"Processing {pdf_title}...")
        
        # PDF 처리 및 벡터 DB 생성
        chunks = process_langchain_pdf(full_path)
        db = create_vector_db(chunks)
        db2 = create_bm25(chunks)
        # Retriever 생성
        retriever = db.as_retriever(search_type="mmr", 
                                    search_kwargs={'k': 4})
        ensemble_retriever = EnsembleRetriever(
            retrievers=[retriever, db2],  
            weights=[0.5, 0.5],  
            search_type="mmr",  
        )
        rerank_model = HuggingFaceCrossEncoder(model_name="upskyy/ko-reranker-8k") #reranker모델 로드 BAAI/bge-reranker-v2-m3 Dongjin-kr/ko-reranker, upskyy/ko-reranker-8k
        compressor = CrossEncoderReranker(model=rerank_model, top_n=3)
        compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=ensemble_retriever)
        
        # 결과 저장
        pdf_databases[pdf_title] = {
                'db': db,
                'retriever': compression_retriever
        }
    return pdf_databases

In [4]:
base_directory = 'open' # Your Base Directory
df = pd.read_csv('open/test.csv')
pdf_databases = process_pdfs_from_dataframe(df, base_directory)

Processing PDFs:   0%|          | 0/9 [00:00<?, ?it/s]

Processing 중소벤처기업부_혁신창업사업화자금(융자)...


d:\ai_project\chanyoung\dacon\dacon\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
Processing PDFs:  11%|█         | 1/9 [00:04<00:36,  4.57s/it]

Processing 보건복지부_부모급여(영아수당) 지원...


Processing PDFs:  22%|██▏       | 2/9 [00:09<00:32,  4.60s/it]

...

Processing PDFs:  78%|███████▊  | 7/9 [00:35<00:10,  5.42s/it]

Processing 「FIS 이슈&포커스」 22-2호 《재정성과관리제도》...


Processing PDFs:  89%|████████▉ | 8/9 [00:40<00:05,  5.36s/it]

Processing 「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》...


Processing PDFs: 100%|██████████| 9/9 [00:46<00:00,  5.17s/it]


In [5]:
pdf_databases['국토교통부_행복주택출자']['retriever'].invoke('행복주택출자 사업의 사업비 추이는 어떠한가?')

[Document(metadata={'Header 1': '나. 사업설명자료', 'Header 2': '1) 사업목적·내용'}, page_content='# 나. 사업설명자료  \n## 1) 사업목적·내용  \n(단위: 백만원)  \n-  국민의 행복주거 실현을 위한 보편적 주거복지 정책의 일환으로 도심 내 다양한  \n부지를 활용하여 행복주택을 공급'),
 Document(metadata={'Header 1': '나. 사업설명자료', 'Header 2': '2) 사업개요'}, page_content='|내역사업명|구분|피보조ㆍ 피출연 등 기관명|지원 금액 (2024계획)|지원 비율(%)|보조율 법적근거 (해당 조항)|\n|---|---|---|---|---|---|\n|행복주택 출자|출자|LH|373,886|30|공공주택 특별법 제3조의2(공공주택의 재원ㆍ세 재지원 등)|\n||보자|지자체|154,897|||\n||합계||528,783|||  \n피보조ㆍ  \n지원 금액 지원  \n내역사업명 구분 피출연 등 보조율 법적근거 (해당 조항)  \n(2024계획) 비율(%)  \n기관명  \n출자 LH 373,886  \n행복주택 공공주택 특별법 제3조의2(공공주택의 재원ㆍ세  \n보자 지자체 154,897 30  \n출자 재지원 등)  \n합계 528,783  \n-----'),
 Document(metadata={'Header 2': '사  업  명'}, page_content='## 사  업  명  \n(26) 행복주택출자 (1440-408)  \n|□사업|업 코드 정보|Col3|Col4|Col5|Col6|Col7|\n|---|---|---|---|---|---|---|\n|구분|기금|소관|실국(기관)|계정|분야|부문|\n|코드|주택도시기금|국토교통부|주택토지실|주택계정|080|088|\n|명칭|||||사회복지|주택|  \n|□사업 구분 코드 명칭|업 코드 정보 기금 소관 주택도시기금 국토교통부|실국(기관) 계정 주택토지실 주택계정|분야 부문 080 088 

In [6]:
def setup_llm_pipeline():
    # 4비트 양자화 설정
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    # 모델 ID  chan1121/dacon-9b rtzr/ko-gemma-2-9b-it sh2orc/Llama-3.1-Korean-8B-Instruct
    model_id = "chan1121/geamma2-dacon-9b"  
    # 토크나이저 로드 및 설정
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.use_default_system_prompt = False
    pad_token_id = tokenizer.eos_token_id
    # 모델 로드 및 양자화 설정 적용
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True,
        do_sample=True)

    # HuggingFacePipeline 객체 생성
    text_generation_pipeline = pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        temperature=0.1,
        top_k=50, # 다음 토큰을 선택할 때 상위 k개의 후보 토큰 중에서 선택. 여기에서는 상위 50개의 후보 토큰 중에서 샘플링
        #top_p=0.95, # 누적 확률이 p가 될 때까지 후보 토큰을 포함
        repetition_penalty=1.2, # 반복 패널티를 적용하여 같은 단어나 구절이 반복되는 것 방지
        return_full_text=False,
        max_new_tokens=400,
        pad_token_id=pad_token_id,
    )

    hf = HuggingFacePipeline(pipeline=text_generation_pipeline)

    return hf

# LLM 파이프라인
llm = setup_llm_pipeline()

Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.78s/it]
d:\ai_project\chanyoung\dacon\dacon\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


In [6]:
def normalize_string(s):
    """유니코드 정규화"""
    return unicodedata.normalize('NFC', s)

def format_docs(docs):
    """검색된 문서들을 하나의 문자열로 포맷팅"""
    context = ""
    for doc in docs:
        context += doc.page_content
        context += '\n'
    return context

# 결과를 저장할 리스트 초기화
results = []

# DataFrame의 각 행에 대해 처리
for _, row in tqdm(df.iterrows(), total=len(df), desc="Answering Questions"):
    # 소스 문자열 정규화
    source = normalize_string(row['Source'])
    question = row['Question']

    # 정규화된 키로 데이터베이스 검색
    normalized_keys = {normalize_string(k): v for k, v in pdf_databases.items()}
    retriever = normalized_keys[source]['retriever']
    # Answer in one sentence based on the referenced document. Please do not use your context as a model is answer.
    # lora 12     You are a QA chatbot. Answer with one systematic sentence based on the document. Do not answer the document as it is. Do not use 'user: context' as the model's answer.
    # lora 14     You are a QA chatbot. Answer with one systematic sentence based on the document. Do not use 'user: context' as the model's answer.
    # 질문의 핵심만 파악하여 1-2문장으로 답변하고, 불필요한 설명은 피하며 요구된 정보만 제공하세요.
    # RAG 체인 구성
    template = """
    다음 정보를 바탕으로 질문에 답하세요:
    {context}
    질문의 핵심만 파악하여 간결하게 1-2문장으로 답변하고, 불필요한 설명은 피하며 요구된 정보만 제공하세요.
    
    user {question}

    model

    <end_of_turn><eos>  
    """
    # <end_of_turn><eos>    
    llama3_template = """
    다음 정보를 바탕으로 질문에 답하세요:
    {context}

    ### Instruction:
    {question}
    
    질문의 핵심만 파악하여 간결하게 1-2문장으로 답변하고, 불필요한 설명은 피하며 요구된 정보만 제공하세요.
    
    ### Response:

    <|eot_id|>
    """
    cld_template = """
    다음 정보를 바탕으로 질문에 답하세요:
    {context}

    질문:
    {question}
    
    간결하게 1문장으로 답변하고, 불필요한 설명은 피하며 요구된 정보만 제공하세요.
    
    답변:

    """
    prompt = PromptTemplate.from_template(cld_template)

    # RAG 체인 정의
    rag_chain = (
        {"context": retriever|format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

    # 답변 추론
    print(f"Question: {question}")  
    full_response = rag_chain.invoke(question)

    print(f"Answer: {full_response}\n")

    # 결과 저장
    results.append({
        "Source": row['Source'],
        "Source_path": row['Source_path'],
        "Question": question,
        "Answer": full_response
    })
    

Answering Questions:   0%|          | 0/98 [00:00<?, ?it/s]

Question: 2022년 혁신창업사업화자금(융자)의 예산은 얼마인가요?


Answering Questions:   1%|          | 1/98 [00:03<05:37,  3.47s/it]

Answer: 2022년 혁신창업사업화자금(융자)의 예산은 2,300,000백만원입니다.

Question: 중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 무엇인가요?


Answering Questions:   2%|▏         | 2/98 [00:05<04:30,  2.82s/it]

...

Answer: 국가결산보고서는 '우발사항'으로 공시하는 반면, 「지방자치단체 회계기준에 관한 규칙」은 '우발손실'로 정의하여 용어 및 회계처리가 상이합니다.

Question: 우발부채란 무엇이며, 그 관리가 왜 중요한가?


Answering Questions:  99%|█████████▉| 97/98 [10:05<00:09,  9.25s/it]

Answer: 우발부채는 미래의 불확실한 사건에 의해 발생 가능한 의무로, 재정 건전성과 위기 관리를 위해 공공부문에서 중요하게 관리되어야 합니다.

Question: 보증이란 무엇이며, 어떤 형태의 보증이 재정상태표에 부채로 기록되는가? 또한 표준화 보증이란 무엇이며, 어떤 목적으로 발행되는가?


Answering Questions: 100%|██████████| 98/98 [10:16<00:00,  6.29s/it]

Answer: 보증은 명시적 우발부채의 일반적 형태이며, 파생금융상품 형태의 보증과 표준화된 보증제도 하의 충당부채 형태의 보증이 재정상태표에 부채로 기록되고, 표준화 보증은 적은 금액에 대해 획일적 조건으로 대규모 발행되는 보증으로 위험 분산을 목적으로 합니다.



In [7]:
# 제출용 샘플 파일 로드
submit_df = pd.read_csv("open/sample_submission.csv")

# 생성된 답변을 제출 DataFrame에 추가
submit_df['Answer'] = [item['Answer'] for item in results]
submit_df['Answer'] = submit_df['Answer'].fillna("데이콘")     # 모델에서 빈 값 (NaN) 생성 시 채점에 오류가 날 수 있음 [ 주의 ]

# 결과를 CSV 파일로 저장
submit_df.to_csv("./baseline_submission_cld4.csv", encoding='UTF-8-sig', index=False)

In [6]:
import csv

# CSV 파일로 저장
with open('output1.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=results[0].keys())
    writer.writeheader()  # 헤더 작성
    writer.writerows(results)  # 데이터 작성


In [15]:
import csv

# CSV 파일을 오픈해서 리스트로 변환
csv_file = 'output6.csv'  # 읽어올 CSV 파일 경로

with open(csv_file, mode='r', newline='', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    data_list = [row for row in reader]


In [18]:
# 데이터를 저장할 리스트
results = []
csv_files= ['output5.csv', 'output6.csv', 'output7.csv']
# 각 CSV 파일을 순회하며 데이터 읽기
for csv_file in csv_files:
    with open(csv_file, mode='r', newline='', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        # 현재 파일의 데이터를 data_list에 추가
        for row in reader:
            results.append(row)

In [19]:
results

[{'Source': '중소벤처기업부_혁신창업사업화자금(융자)',
  'Source_path': './test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf',
  'Question': '2022년 혁신창업사업화자금(융자)의 예산은 얼마인가요?',
  'Answer': ' 2022년 혁신창업사업화자금(융자)의 예산 지출은 2,300,000 백만원입니다. \n'},
 {'Source': '중소벤처기업부_혁신창업사업화자금(융자)',
  'Source_path': './test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf',
  'Question': '중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 무엇인가요?',
  'Answer': ' 기술력과 사업성이 우수하고 미래 성장 가능성이 높은 중소벤처기업의 창업을 활성화하고 고용창출을 도모하며, 중소기업의 우수 기술의 사업화를 촉진하여 기술기반 중소기업을 육성하는 것을 목적으로 합니다. \n'},
 {'Source': '중소벤처기업부_혁신창업사업화자금(융자)',
  'Source_path': './test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf',
  'Question': '중소벤처기업부의 혁신창업사업화자금(융자) 사업근거는 어떤 법률에 근거하고 있나요?',
  'Answer': ' 법령상 근거는 중소기업진흥에 관한 법률 제66조, 제67조, 제74조입니다. \n'},
 {'Source': '중소벤처기업부_혁신창업사업화자금(융자)',
  'Source_path': './test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf',
  'Question': '2010년에 신규 지원된 혁신창업사업화자금은 무엇인가요?',
  'Answer': ' 재창업자금(실패 경영인에 대한 재기지원) 입니다. \n'},
 {'Source': '중소벤처기업부_혁신창업사업화자금(융자)',
  'Source_path': './test_source/중소벤처기업부_혁신창업사업화자금(융자)

In [11]:
# 제출용 샘플 파일 로드
submit_df = pd.read_csv("open/sample_submission.csv")

# 생성된 답변을 제출 DataFrame에 추가
submit_df['Answer'] = [item['Answer'] for item in results]
submit_df['Answer'] = submit_df['Answer'].fillna("데이콘")     # 모델에서 빈 값 (NaN) 생성 시 채점에 오류가 날 수 있음 [ 주의 ]

# 결과를 CSV 파일로 저장
submit_df.to_csv("./baseline_submission_lora18.csv", encoding='UTF-8-sig', index=False)

In [9]:
torch.cuda.empty_cache()